##**Installations**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!unzip /content/drive/MyDrive/ASR.zip

In [3]:
%%capture
# !pip show kenlm
import os

!pip install fairseq
!pip install 'https://github.com/kpu/kenlm/archive/master.zip'
!pip install pykaldi

os.environ['FAIRSEQ_ROOT'] = ' /usr/local/lib/python3.7/dist-packages/fairseq'
os.environ['KALDI_ROOT'] = ' /usr/local/lib/python3.7/dist-packages/kaldi'
os.environ['KENLM_ROOT'] = ' /usr/local/lib/python3.7/dist-packages/kenlm'

!git clone 'https://github.com/zhenghuatan/rVADfast.git'
os.environ['RVAD_ROOT'] = '/content/rVADfast'

In [4]:
%%capture
%mkdir /content/trial/
%mkdir /content/trial/audio/
%mkdir /content/trial/gen/
!tar -xvzf /content/drive/MyDrive/dev-clean.tar.gz -C /content/trial/audio/

##**LibriSpeech**

Create Training and Validation Split from aw Audio

In [5]:
!python /usr/local/lib/python3.7/dist-packages/fairseq/examples/wav2vec/wav2vec_manifest.py /content/trial/audio/LibriSpeech/dev-clean/ --ext 'flac' --dest /content/trial/ --valid-percent 0.95

In [6]:
%cd ASR

/content/ASR


Voice activity detection to remove silence

In [7]:
# df=pd.read_csv('/content/trial/train.tsv', delimiter='\t')
# df.to_csv('/content/trial/train.tsv', index=False, sep='\t')
!python scripts/vads.py -r '/content/rVADfast' <'/content/trial/train.tsv' >'train.vads'

100% 143/143 [04:48<00:00,  2.02s/it]


Remove silence

In [8]:
!python scripts/remove_silence.py --tsv '/content/trial/train.tsv' --vads 'train.vads' --out  '/content/trial/gen/'

100% 143/143 [00:01<00:00, 100.41it/s]


Download wav2vec2.0 model

In [9]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 880 kB 46.1 MB/s 
     |████████████████████████████████| 6.6 MB 34.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0a9e6fce1cef92aad993eb3ccc5e644da623d44f465e85031ba368deee325226
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 43.8 MB/s 
     |████████████████████████████████| 212 kB 39.3 MB/s 
     |████████████████████████████████| 136 kB 49.2 MB/s 
     |████████████████████████████████| 127 kB 46.8 MB/s 
     |████████████████████████████████| 271 kB 46.2 MB/s 
     |████████████████████████████████| 144 kB 32.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
  Attempting 

In [10]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/843 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# print(model)
torch.save(model.state_dict(), '/content/trial/model.pt')

Preprocess audio

In [12]:
%mkdir /content/trial/output_int1/
!bash /content/ASR/scripts/prepare_audio.sh /content/trial /content/trial/output_int1 /content/trial/model.pt 512 14

using 512 dim for PCA
extracting from layer 14
processing splits: train
cp: cannot stat '/content/trial/*.wrd': No such file or directory
cp: cannot stat '/content/trial/*.ltr': No such file or directory
cp: cannot stat '/content/trial/*.phn': No such file or directory
cp: cannot stat '/content/trial/dict*': No such file or directory
/content/ASR/scripts/prepare_audio.sh: line 53: setopt: command not found
python3: can't open file '/usr/local/lib/python3.7/dist-packages/fairseq/examples/wav2vec/unsupervised/scripts/wav2vec_extract_features.py': [Errno 2] No such file or directory
python3: can't open file '/usr/local/lib/python3.7/dist-packages/fairseq/examples/wav2vec/unsupervised/scripts/wav2vec_cluster_faiss.py': [Errno 2] No such file or directory
python3: can't open file '/usr/local/lib/python3.7/dist-packages/fairseq/examples/wav2vec/unsupervised/scripts/wav2vec_apply_cluster_faiss.py': [Errno 2] No such file or directory
python3: can't open file '/usr/local/lib/python3.7/dist-pac

Preprocess words to phonemes as true samples to feed the generator

In [13]:
%mkdir /content/trial/words
!bash scripts/prepare_text.sh english /content/drive/MyDrive/file.txt /content/trial/words/ 1000 espeak /content/drive/MyDrive/lid.176.bin

english
english
/content/drive/MyDrive/file.txt
/content/trial/words/
min phone seen threshold is 1000
grep: /content/trial/words//lm.upper.lid.txt: No such file or directory
python3: can't open file '/usr/local/lib/python3.7/dist-packages/fairseq/examples/wav2vec/unsupervised/scripts/normalize_and_filter_text.py': [Errno 2] No such file or directory
python3: can't open file '/usr/local/lib/python3.7/dist-packages/fairseq/fairseq_cli/preprocess.py': [Errno 2] No such file or directory
cut: /content/trial/words//dict.txt: No such file or directory
grep: /content/trial/words//words.txt: No such file or directory
scripts/prepare_text.sh: line 52: /content/trial/words//words.txt: No such file or directory
paste: /content/trial/words//words.txt: No such file or directory
python3: can't open file '/usr/local/lib/python3.7/dist-packages/fairseq/fairseq_cli/preprocess.py': [Errno 2] No such file or directory
scripts/prepare_text.sh: line 65: /content/trial/words//lexicon.lst: No such file or d

#**TIMIT**
Load and preprocess

In [14]:
%mkdir /content/TIMIT/
!unzip /content/drive/MyDrive/TIMIT.zip -d /content/TIMIT/

Streaming output truncated to the last 5000 lines.
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SX251.PHN  
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SX307.PHN  
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SI701.TXT  
 extracting: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SA1.TXT  
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SX341.WAV  
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SX431.WRD  
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SX71.WRD  
 extracting: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SI2148.TXT  
 extracting: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SI1331.TXT  
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SX251.WRD  
  inflating: /content/TIMIT/data/lisa/data/timit/raw/TIMIT/TRAIN/DR4/MJAC0/SX307

In [15]:
%mkdir /content/timitop

In [16]:
!pwd

/content/ASR


In [17]:
!bash /content/ASR/scripts/prepare_timit.sh '/content/TIMIT/data/lisa/data/timit/raw/TIMIT/'  '/content/timitop/' '/content/model.pt'

awk: cannot open -v (No such file or directory)
